# [![](https://img.shields.io/badge/Video-Hướng%20dẫn-ff0000)](https://youtu.be/798qchqWty4) [![](https://img.shields.io/badge/Design-stablediffusion.vn-0075ff)](https://stablediffusion.vn) [![](https://img.shields.io/badge/Ver-2.2-0075ff)](https://stablediffusion.vn/update/) [![](https://img.shields.io/badge/All%20Tools-sdvn.me-0075ff)](https://stablediffusion.vn/bo-cong-cu/) [![](https://img.shields.io/badge/SDVN-Library-green)](https://bit.ly/sdvn-lib) [![](https://img.shields.io/badge/Khoá%20học-SD%20Flux-red)](https://hungdiffusion.com/) [![](https://img.shields.io/badge/Group-Support-0075ff)](https://www.facebook.com/groups/stablediffusion.vn) [![](https://img.shields.io/discord/813085864355037235?color=blue&label=Discord&logo=Discord)](https://discord.gg/5SEtApPeyG)
---
🧨 `UI Colab - A100, UI Gradio - L4/A100`

🧨 `Thêm Caption API cho Gemini và ChatGPT`

🧨 `Bật lại chức năng Sampler khi train`

#☕️ 1.Cài đặt

In [ ]:
#@title ✨ Cài đặt UI

# @markdown 🟡 `Lựa chọn Gradio nếu muốn train trực tiếp với giao diện UI (Sử dụng link gradio) `
%cd /content
from google.colab import drive
drive.mount('/content/drive')

UI = 'Colab' # @param ['Gradio','Colab']
UpdateRepo = False # @param {"type":"boolean"}

root_dir = "/content"
repo_dir = f"{root_dir}/SDVN-training-colab-flux"
toolkit_dir = f"{repo_dir}/ai-toolkit"
config_dir = f"{repo_dir}/Config"
data_dir = f"/content/drive/MyDrive/SD-Data"

%cd {root_dir}
import os
if not os.path.exists(repo_dir):
  !git clone  --recurse-submodules https://github.com/StableDiffusionVN/SDVN-training-colab-flux {repo_dir}
  %env TF_CPP_MIN_LOG_LEVEL=1
  %env LD_PRELOAD={repo_dir}/libtcmalloc_minimal.so.4

if UpdateRepo:
  %cd {toolkit_dir}
  !git checkout main
  !git pull
  !pip install -q -r /content/SDVN-training-colab-flux/ai-toolkit/requirements.txt
  !pip install OpenAI google-genai
else:
  !pip install -q -r {repo_dir}/requirements.txt

%run {repo_dir}/TrainScript.ipynb

if UI == "Gradio":
  !rm -r {toolkit_dir}/output
  !ln -s {data_dir}/Lora {toolkit_dir}/output
  !python {toolkit_dir}/flux_train_ui.py


#⏳ 2. Cấu hình train

In [ ]:
#@title ✨ 2.1 Xử lý dữ liệu
#@markdown [![](https://img.shields.io/badge/Caption-Tools-blue)](https://colab.research.google.com/github/StableDiffusionVN/SDVN-training-colab-flux/blob/main/SDVN_Caption_Tools.ipynb)

# @markdown 🟡 `Train nhiều lora bằng cách nhập nhiều thư mục, cách nhau bằng dấu phẩy (,)`
TrainFolder = "/content/drive/MyDrive/SD-Data/TrainData"  # @param {type:'string'}
DataClean = False #@param {type:"boolean"}
No_gen_caption = True #@param {type:"boolean"}
# @markdown 🟡 `OCR Prompt`
Caption = 'Florence' # @param ['None','Blip','Florence', 'Joy_Caption', 'APIGemini | 2.0 Flash', 'APIGemini | 2.0 Flash Lite', 'APIOpenAI | GPT 4-o mini']
APIkey = "" # @param {type:'string'}
Caption_Length = "Medium" # @param ["Short","Medium","Long"]
# @markdown 🟡 `Joy Caption Extra`
Joy_Type = "Training Prompt" # @param ["Descriptive", "Descriptive (Informal)", "Training Prompt", "MidJourney", "Booru tag list", "Booru-like tag list", "Art Critic", "Product Listing", "Social Media Post"]
Joy_Extra_Option = "" # @param ["","If there is a person/character in the image you must refer to them as {name}.","Do NOT include information about people/characters that cannot be changed (like ethnicity, gender, etc), but do still include changeable attributes (like hair style).","Include information about lighting.","Include information about camera angle.","Include information about whether there is a watermark or not.","Include information about whether there are JPEG artifacts or not.","If it is a photo you MUST include information about what camera was likely used and details such as aperture, shutter speed, ISO, etc.","Do NOT include anything sexual; keep it PG.","Do NOT mention the image's resolution.","You MUST include information about the subjective aesthetic quality of the image from low to very high.","Include information on the image's composition style, such as leading lines, rule of thirds, or symmetry.","Do NOT mention any text that is in the image.","Specify the depth of field and whether the background is in focus or blurred.","If applicable, mention the likely use of artificial or natural lighting sources.","Do NOT use any ambiguous language.","Include whether the image is sfw, suggestive, or nsfw.","ONLY describe the most important elements of the image."] {"allow-input":true}
Joy_Character_Name = "" # @param {type:'string'}
Joy_Custom_Prompt = "" # @param {type:'string'}
# @markdown 🟡 `API Caption`
API_Prompt = "" # @param {type:'string'}
# @markdown 🟡 `Thêm caption tuỳ chọn`
Custom_Caption = "" # @param {type:'string'}
AddFolderName = False #@param {type:"boolean"}
Remove_Caption = False #@param {type:"boolean"}
Append = False #@param {type:"boolean"}

if No_gen_caption == True:
  Caption = 'None'
  Custom_Caption = ''
  AddFolderName = False
extension =".txt"
train_folder_list = TrainFolder.split(",")

delete_specific_models()
if Caption == 'Joy_Caption':
  #Load Joy_Model
  # Load CLIP
  !git clone https://huggingface.co/spaces/fancyfeast/joy-caption-alpha-two /content/joy-caption-alpha-two

  print("Loading CLIP")
  clip_processor = AutoProcessor.from_pretrained(CLIP_PATH)
  clip_model = AutoModel.from_pretrained(CLIP_PATH)
  clip_model = clip_model.vision_model

  assert (CHECKPOINT_PATH / "clip_model.pt").exists()
  print("Loading VLM's custom vision model")
  checkpoint = torch.load(CHECKPOINT_PATH / "clip_model.pt", map_location='cpu')
  checkpoint = {k.replace("_orig_mod.module.", ""): v for k, v in checkpoint.items()}
  clip_model.load_state_dict(checkpoint)
  del checkpoint

  clip_model.eval()
  clip_model.requires_grad_(False)
  clip_model.to("cuda")

  # Tokenizer
  print("Loading tokenizer")
  tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT_PATH / "text_model", use_fast=True)
  assert isinstance(tokenizer, PreTrainedTokenizer) or isinstance(tokenizer, PreTrainedTokenizerFast), f"Tokenizer is of type {type(tokenizer)}"

  # LLM
  print("Loading LLM")
  print("Loading VLM's custom text model")
  text_model = AutoModelForCausalLM.from_pretrained(CHECKPOINT_PATH / "text_model", device_map=0, torch_dtype=torch.bfloat16)
  text_model.eval()

  # Image Adapter
  print("Loading image adapter")
  image_adapter = ImageAdapter(clip_model.config.hidden_size, text_model.config.hidden_size, False, False, 38, False)
  image_adapter.load_state_dict(torch.load(CHECKPOINT_PATH / "image_adapter.pt", map_location="cpu"))
  image_adapter.eval()
  image_adapter.to("cuda")

Cap_prompt = {
    'Short':['<CAPTION>',10,30,'short'],
    'Medium':['<DETAILED_CAPTION>',10,100,'medium-length'],
    'Long':['<MORE_DETAILED_CAPTION>',10,150,'very long']
}

for folder in train_folder_list:
  if DataClean == True :
    %cd /content
    clean_directory(folder)
  if Caption != "None":
    caption_dir(folder,Cap_prompt[Caption_Length][0])
  if AddFolderName:
    add_forder_name(folder)
  if Custom_Caption != "":
    process_dir(folder, Custom_Caption, Append, Remove_Caption)

delete_specific_models()

In [ ]:
#@title ⚙️ 2.2 Cài đặt train
Low_VRAM = False
Lora_name = "lora_name" # @param {"type":"string","placeholder":"Tên lora"}
OutputFolder = "/content/drive/MyDrive/SD-Data/Lora" #@param {type:'string'}
# @markdown 🟡 `Điều chỉnh steps theo mức độ train mong muốn, các thư mục đặt tên theo cú pháp {steps}_{name} sẽ tự nhận steps riêng theo tên`
Steps = 1000 #@param {type:'number'}
Save_steps = 200 #@param {type:'number'}
Batch_size = 4 #@param {type:'number'}
# @markdown 🟡 `Các thông số nên để theo mặc định`
Lr = 1e-4 #@param {type:'number'}
Dim = 16 #@param {type:'number'}
Alpha = 16 #@param {type:'number'}
Resolution = "512, 768, 1024" #@param {type:'string'}
Train_TexEncoder = False
Sampler_Prompt = "" #@param {type:'string'}
Sampler_Steps = 100 #@param {type:'number'}

folder_train = []
for folder in train_folder_list:
  if len(get_supported_images(folder)) > 0:
    folder_dic = {
    "path": folder,
    "steps": get_steps(folder)[0],
    "name": get_steps(folder)[1].replace(" ", "_")
    }
    folder_train.append(folder_dic)
  else:
    print(f"Thư mục [ {folder} ] có thể không chứa ảnh được hỗ trợ, hãy kiểm tra lại (.png, .jpg, .jpeg, .webp, .bmp, .JPG, .JPEG, .PNG)")

for folder in folder_train:
  config(folder)


#3. 🧨 Train

In [ ]:
#@title ⏳ Run Lora Train
AutoDisconect = False #@param {type:"boolean"}
train()
if AutoDisconect:
    from time import sleep
    sleep(180)
    from google.colab import runtime
    runtime.unassign()